## Decision models homework 5, yee-haw!

In [1]:
# Rustle up some asynchronous tensor manipulation, partner!
import jax
from jax import numpy as np

In [2]:
import grub

/data/home/will/.conda/envs/will-gpu/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')
/data/home/will/.conda/envs/will-gpu/lib/python3.8/site-packages/jax/lax/lax.py:6198: UserWarning: Explicitly requested dtype int64 requested in array is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))


In [3]:
# Rustle up some grub there, partner!
cowboy_details = grub.rustle()

# add joke about horse
clue_about_maiden_whereabouts = jax.random.PRNGKey(1)
lookin_for_maidens = jax.random.uniform(clue_about_maiden_whereabouts, shape=(cowboy_details.n_cowboys, 20), dtype=jax.numpy.float16)
maiden_mood = jax.random.normal(clue_about_maiden_whereabouts, shape=(cowboy_details.n_cowboys, 20), dtype=jax.numpy.float16)

In [4]:
# Dont squat with your spurs on!

@jax.jit
def rescue_maiden(luck, maiden_mood):
    did_we_rescue_the_maiden = luck > cowboy_details.cowgirl_needs_rescuing_prob * cowboy_details.cowgirl_rescued_prob
    maiden_gratitude = cowboy_details.gratitude_mean + cowboy_details.gratitude_std*maiden_mood

    adjusted_gratitude = maiden_gratitude*did_we_rescue_the_maiden
    
    return np.sum(adjusted_gratitude)
    
    
@jax.jit
def initiate_adventure(do_luck_be_on_our_side, is_it_that_time_of_the_month):
    return jax.vmap(rescue_maiden)(do_luck_be_on_our_side, is_it_that_time_of_the_month)

In [5]:
our_adventure_was_great = initiate_adventure(do_luck_be_on_our_side=lookin_for_maidens, is_it_that_time_of_the_month = maiden_mood)

In [6]:
we_rescued_many = our_adventure_was_great.std() / (len(our_adventure_was_great)**(1/2))

# thanks google 
we_rescued_many *= 1.96

In [7]:
print(our_adventure_was_great.mean()+we_rescued_many)
print(our_adventure_was_great.mean()-we_rescued_many)

412.21704
410.10364
